In [1]:
import pandas as pd
import numpy as np
import gc
import pickle
np.random.seed(1)

import nltk
from nltk.corpus import stopwords
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

#Yep no stemmer

In [2]:
%%time
train = pd.read_csv('avito-text-classification/train.csv')
test = pd.read_csv('avito-text-classification/test.csv')
train = train.fillna("")
test = test.fillna("")
train["title_description"] = train[["title", "description"]].agg(" ".join, axis=1)
test["title_description"] = test[["title", "description"]].agg(" ".join, axis=1)
train = train.drop(["title", "description"], axis=1)
test = test.drop(["title", "description"], axis=1)
gc.collect()

Wall time: 3min 32s


0

In [3]:
def my_tokenizer(words):
    stop_words = set(stopwords.words("russian")) 
    return [w for w in nltk.word_tokenize(str(words)) if not ((w in stop_words) or (w[0] in string.punctuation) or (w[0].isnumeric()))] 

In [4]:
%%time
try:
    with open('countvect1.pkl', 'rb') as fin:
        cv = pickle.load(fin)
except:
    with open("countvect1.pkl", 'wb') as fout:
        cv = CountVectorizer(tokenizer=my_tokenizer, ngram_range=(1, 2))
        cv.fit(train["title_description"])
        pickle.dump(cv, fout)
gc.collect()

Wall time: 17.4 s


40

In [5]:
%%time
try:
    with open('x_counts1.pkl', 'rb') as fin:
        X_counts = pickle.load(fin)
except:
    with open("x_counts1.pkl", 'wb') as fout:
        X_counts = cv.transform(train["title_description"])
        pickle.dump(X_counts, fout)

        
            
print(X_counts.shape)
gc.collect()

(4234042, 35124132)
Wall time: 850 ms


20

In [6]:
%%time
try:
    with open('model_1.pkl', 'rb') as fin:
        model = pickle.load(fin)
except FileNotFoundError:
    with open("model_1.pkl", 'wb') as fout:
        model = LinearSVC(max_iter=1000000, class_weight="balanced", C=0.1)
        model.fit(X_counts, train["Category"])
        pickle.dump(model, fout)
gc.collect()

Wall time: 8h 8min 16s


20

In [7]:
%%time
y_pred = model.predict(X_counts)
acc = accuracy_score(train["Category"], y_pred)
print(acc)
gc.collect()

0.980310776322011
Wall time: 28.1 s


20

In [8]:
%%time
try:
    with open('x_test1.pkl', 'rb') as fin:
        X_tests = pickle.load(fin)
except FileNotFoundError:
    with open("x_test1.pkl", 'wb') as fout:
        X_tests = cv.transform(test["title_description"])
        pickle.dump(X_tests, fout)
output = model.predict(X_tests)
data = pd.DataFrame()
data["Id"] = test["itemid"]
data["Category"] = pd.Series(output)
data.to_csv("prediction.csv", index=False)

Wall time: 10.6 s


In [9]:
del test
del train
gc.collect()

20